In [8]:
import torch
import torchvision as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim


In [9]:
import torch
import torchvision
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import SGD

# Path to your dataset directory
dataset_path = './trainingData'
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images to 224x224 (adjust as needed)
    transforms.ToTensor(),  # Convert the images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the images
])
# Load the dataset
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)  # Adjust batch size as needed

# Define the model (example using ResNet18)
model = models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 8)  # Adjust the output layer to match the number of classes

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training the model
num_epochs = 5  # Set the number of epochs
model.train()  # Set the model to training mode

for epoch in range(num_epochs):

    total_loss = 0
    for inputs, labels in dataloader:

        optimizer.zero_grad()  # Zero the parameter gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize

        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}')
    torch.save(model.state_dict(), 'model_state_dict.pth')



c:\Users\chaha\OneDrive\Desktop\mlcatproject\env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\chaha\OneDrive\Desktop\mlcatproject\env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1, Loss: 1.4474541544914246
Epoch 2, Loss: 0.8290991502649644
Epoch 3, Loss: 0.6196648180484772
Epoch 4, Loss: 0.4704562749932794
Epoch 5, Loss: 0.4079539432537322


In [10]:
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(num_ftrs, 8)  # Re-establish the modified fully connected layer
model.load_state_dict(torch.load('model_state_dict.pth'))

<All keys matched successfully>

In [11]:
import torch
from torchvision import transforms
from PIL import Image

# Function to load and transform an image
def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize the image to what the model expects
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    # Add an extra batch dimension since PyTorch treats all inputs as batches
    image = transform(image).unsqueeze(0)
    return image

# Path to your image
image_path = './real3.jpg'

# Load and transform the image
image = load_image(image_path)

# Set the model to evaluation mode
model.eval()

# No gradient is needed to be computed
with torch.no_grad():
    output = model(image)
    # Convert output probabilities to predicted class (assuming you have 8 classes)
    probabilities = torch.nn.functional.softmax(output, dim=1)
    predicted_class = probabilities.argmax(dim=1)

class_labels = ['Airplane', 'Bike', 'Boat', 'motorbus', 'motorcycle', 'seaplane', 'train', 'truck']

# Print probabilities for each class
for idx, prob in enumerate(probabilities.squeeze(0)):
    print(f'{class_labels[idx]}: {prob.item():.4f}')

print(f'\nPredicted class: {class_labels[predicted_class.item()]} with highest probability {probabilities.max().item():.4f}')

Airplane: 0.0000
Bike: 0.0015
Boat: 0.0000
motorbus: 0.0000
motorcycle: 0.9985
seaplane: 0.0000
train: 0.0000
truck: 0.0000

Predicted class: motorcycle with highest probability 0.9985


In [12]:
test_dataset = datasets.ImageFolder(root='./testingData', transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)
model.eval()  # Set the model to evaluation mode

correct = 0
total = 0
with torch.no_grad():
    for inputs,labels in test_dataloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)  # Get the predicted classes
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuracy of the model on the test images: {accuracy * 100:.2f}%')

Accuracy of the model on the test images: 78.43%


In [13]:
from sklearn.metrics import classification_report

# Set the model to evaluation mode
model.eval()

# Create empty lists to store the true labels and predicted labels
true_labels = []
predicted_labels = []

# No gradient is needed to be computed
with torch.no_grad():
    for inputs, labels in test_dataloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)  # Get the predicted classes
        
        true_labels.extend(labels.tolist())
        predicted_labels.extend(predicted.tolist())

# Generate the classification report
report = classification_report(true_labels, predicted_labels)

print(report)


              precision    recall  f1-score   support

           0       0.97      0.67      0.80       101
           1       0.88      0.92      0.90        75
           2       0.78      0.95      0.85       133
           3       0.56      0.38      0.45        64
           4       0.68      0.99      0.81       147
           5       0.88      0.89      0.88        71
           6       0.79      0.51      0.62        61
           7       0.82      0.69      0.75       164

    accuracy                           0.78       816
   macro avg       0.80      0.75      0.76       816
weighted avg       0.80      0.78      0.77       816

